In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [14]:
import utils
word = 'горшок'
weights = utils.load_weights('../corpora/ad-nouns/cdict/', word)
texts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', word)
texts = [[w for w in ctx if weights.get(w, 0) > 1] for ctx in texts]
texts = [ctx for ctx in texts if ctx]
for text in texts[:3]:
    print(text)
print(len(texts))
#texts = texts[:1000]

['золото', 'почва']
['черепок', 'глиняный']
['орхидея', 'цветок', 'поддон', 'класть', 'гравий', 'сверху', 'ставить', 'орхидея']
32223


In [15]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0], corpus[1])

[(0, 1), (1, 1)] [(2, 1), (3, 1)]


In [16]:
#from gensim.models.ldamulticore import LdaMulticore as LdaModel
from gensim.models import LdaModel
print(len(corpus))
lda = LdaModel(corpus, id2word=dictionary, num_topics=6, passes=3, alpha='auto')

32223


In [18]:
lda.print_topics(num_words=5)

[(0,
  '0.046*глиняный + 0.046*стоять + 0.036*окно + 0.027*цветок + 0.022*цветочный'),
 (1,
  '0.076*растение + 0.053*цветок + 0.018*корень + 0.015*высаживать + 0.014*расти'),
 (2, '0.068*вода + 0.025*золотой + 0.021*поставлять + 0.017*см + 0.016*лист'),
 (3,
  '0.039*обжигать + 0.025*мед + 0.025*глина + 0.023*кувшин + 0.017*глиняный'),
 (4, '0.082*ночной + 0.049*цветочный + 0.029*стол + 0.028*каша + 0.021*пол'),
 (5,
  '0.107*ребенок + 0.066*сидеть + 0.045*ходить + 0.037*садиться + 0.034*детский')]

In [19]:
import rl_wsd_labeled
senses, contexts = rl_wsd_labeled.get_contexts(rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))
print(senses)
print(contexts[0])

{'2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)', '1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)', '3': 'Ночной горшок'}
(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.'), '2')


In [20]:
print(contexts[0][0])
print(utils.normalize(contexts[0][0]))

('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.')
['телевизор', 'ковер', 'музыкальный', 'центр', 'стол', 'аквариум', '3', 'шкаф', 'цветок', 'мелкий', 'аксессуар']


In [22]:
import numpy as np
documents = [dictionary.doc2bow(utils.normalize(ctx)) for ctx, _ in contexts]
gamma, _ = lda.inference(documents)
pred_topics = gamma.argmax(axis=1)
true_labels = np.array([int(ans) for _, ans in contexts])
print(pred_topics[:10])
print(true_labels[:10])

[0 1 1 1 2 4 1 2 0 4]
[2 2 2 2 2 3 2 1 2 2]


In [23]:
from sklearn.metrics import v_measure_score, adjusted_rand_score
print('ARI:', adjusted_rand_score(true_labels, pred_topics))
print('V score:', v_measure_score(true_labels, pred_topics))

ARI: 0.27628538129932667
V score: 0.292217376541
